In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
from itertools import product
import datetime
from datetime import date, datetime, timedelta

In [2]:
def xml_reader(url): 
    # URL вашей XML-ссылки
    # Загрузка данных из URL
    response = requests.get(url)
    print(response.status_code)
    print()
    root = ET.fromstring(response.content)

    data = []

    for channel in root.findall('channel'):
            channel_data = {
                'channel_id':channel.get('id'),
                'epgsrvc_id':channel.get('epgsrvc_id'),
                'display_name':channel.find('display-name').text,
                'week':channel.find('week').text,
                'update':channel.find('update').text,
                'link':channel.find('href').text
            }
            data.append(channel_data)
    # Создание DataFrame из списка словарей
    df = pd.DataFrame(data)
    return df

In [3]:
data = xml_reader('http://xmldata.epgservice.ru/EPGService/hs/xmldata/3d6da182-cedb-4315-a685-30294949632/index')

200



In [4]:
url = data.loc[0,'link']
part_link = url.split('/file/')[0] + '/file/'

In [5]:
today = date.today()
start_of_this_week = today - timedelta(days=today.weekday())
start_of_next_week = (start_of_this_week + timedelta(days=7)).strftime('%Y-%m-%d')

weeks = start_of_next_week
channels_list = data['channel_id'].unique()

link_info_list = [
    [i, weeks, f"{part_link}{i}?week={weeks}"]
    for i in product(channels_list)
]

all_links = pd.DataFrame(link_info_list, columns=['id', 'week', 'link'])

In [6]:
all_links = []
for id_channel in channels_list: 
    link = part_link + id_channel + '?=' + start_of_next_week
    all_links.append(link)

In [7]:
len(all_links)

189

In [46]:
def get_text(element, tag):
    """Функция для получения текста из элемента или возврата None, если элемент не найден."""
    found_element = element.find(tag)
    return found_element.text if found_element is not None else None


def pars_programm(result):
    shedule = []
    channel_info = result.find('channel')
    channel = get_text(channel_info, 'display-name') 
    channel_id = channel_info.get('id')

    for programme  in result.findall('programme'):
        start = programme.get('start')
        end = programme.get('stop')
        title = get_text(programme, 'title')
        title_element = programme.find('title')
        title_id = title_element.get('id')
        original_title = get_text(programme, 'original_title')
        category = get_text(programme,'category')

        ganres = [ganre.text for ganre in programme.findall('ganre')]
        topics = [topic.text for topic in programme.findall('topic')]

        credits = programme.find('credits')
        actors = [{person.find('name').text: person.get('type')} for person in credits.findall('person')] if credits is not None else []

        programm_info = [channel, channel_id,  title, title_id, original_title, start, end, category, ganres, topics, actors]
        shedule.append(programm_info)

    return shedule

In [47]:
full_programm = []

for link in all_links:
    response_channel = requests.get(link)
    if response_channel.status_code == 200:
        result_request = ET.fromstring(response_channel.content)
        list_programm = pars_programm(result_request)
        full_programm.extend(list_programm)
    else:
        pass

In [48]:
full_program_df = pd.DataFrame(full_programm, columns = ['channel', 'channel_id',  'title', 'title_id' ,'original_title', 'start', 'end', 'category', 'ganres', 'topics', 'actors'])

In [78]:
programm_shedule = full_program_df[['channel', 'channel_id',  'title', 'title_id' ,'original_title', 'start', 'end', 'category']]

In [133]:
actors = full_program_df[['title_id','actors']].explode('actors')
actors = actors[~actors['actors'].isna()]
actors[['name','role']] = actors['actors'].apply(lambda x: pd.Series(list(x.items())[0]))
actors = actors[['title_id','name','role']].drop_duplicates()

In [92]:
ganres = full_program_df[['title_id','ganres']].explode('ganres')
ganres = ganres[~ganres['ganres'].isna()]
ganres = ganres.drop_duplicates()

In [107]:
topics = full_program_df[['title_id','topics']].explode('topics')
topics = topics[~topics['topics'].isna()]
topics = topics.drop_duplicates()

In [2]:
def convert_time(time_str):
    # Преобразуем строку в объект datetime
    dt = datetime.strptime(time_str, "%Y%m%d%H%M%S %z")
    # Применяем смещение на UTC+5
    dt_utc_plus_5 = dt + timedelta(hours=5)
    return dt_utc_plus_5.strftime("%Y-%m-%d %H:%M:%S")

In [113]:
programm_shedule['start_video'] = programm_shedule['start'].apply(convert_time)
programm_shedule['end_video'] = programm_shedule['end'].apply(convert_time)

C:\Users\m.buranov\AppData\Local\Temp\ipykernel_21332\3295508032.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  programm_shedule['start_video'] = programm_shedule['start'].apply(convert_time)
C:\Users\m.buranov\AppData\Local\Temp\ipykernel_21332\3295508032.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  programm_shedule['end_video'] = programm_shedule['end'].apply(convert_time)


In [118]:
programm_shedule['start_date'] = pd.to_datetime(programm_shedule['start_video']).dt.date

In [122]:
programm_shedule = programm_shedule[['channel', 'channel_id', 'title', 'title_id', 'original_title', 'category', 'start_video', 'end_video', 'start_date']]

In [6]:
# Сохранение файлов в директорию `./` (корневая директория проекта)

programm_path = 'programm_shedule.csv'
programm_shedule.to_csv(programm_path, index=False, encoding='utf-8-sig')

topics_path = 'topics.csv'
topics.to_csv(topics_path, index=False, encoding='utf-8-sig')

ganres_path = 'ganres.csv'
ganres.to_csv(ganres_path, index=False, encoding='utf-8-sig')

actors_path = 'actors.csv'
actors.to_csv(actors_path, index=False, encoding='utf-8-sig')

NameError: name 'programm_shedule' is not defined